In [3]:
import numpy as np
from qva import *
from scipy.linalg import ishermitian
import matplotlib.pyplot as plt
import random
from fem_1d import *
from qva import *
from pauliDecomp import *
from scipy.sparse import diags
import pprint
pp = pprint.PrettyPrinter(width=110, compact=True)
import qiskit.quantum_info as qi
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import AerSimulator
#from qiskit_aer.primitives import Estimator  # import change!!!
from qva import *
from qiskit.primitives import Estimator

In [4]:


###############################
#     <0| U^d An V(k) |0>     #
###############################

nqbits = 2
nlayers = 2

qubits = []
for i in range(0,nqbits):
    qubits.append(i)

## Getting the U operator ##

uc = QuantumCircuit(nqbits)
uc.h(0)
uc.h(1)
U = qi.Operator(uc)
print("uc: ",uc)
print("U: ",U)
print("U dagger: ",U.adjoint())
print("Is the U-Operator Unitary? ",U.is_unitary())
print("Is the U-Operator Hermition? ",ishermitian(U.data))

## Getting the V(k) operator ##

Vcirc = QuantumCircuit(nqbits)
parameters = [1, 2, 3, 4]
Vcirc = ansatz_RYZ(Vcirc, qubits, parameters, nlayers)
print(Vcirc)
V = qi.Operator(Vcirc)
print(Vcirc)
print("V: ",V)
print("Is the V-Operator Unitary? ",V.is_unitary())
print("Is the V-Operator Hermition? ",ishermitian(V.data))

## Getting the An operator ##

An = SparsePauliOp.from_list([("IY", 1)])
An = qi.Operator(An)
print("An: ",An)
print("Is the An-Operator Unitary? ",An.is_unitary())
print("Is the An-Operator Hermition? ",ishermitian(An.data))

## Making the operator ##

op = V.compose(An.compose(U.adjoint()))  # <-- our circuit matches this order
print("Operator: ",op)
print("Is the Operator Unitary? ",op.is_unitary())
print("Is the Operator Hermition? ",ishermitian(op.data))
print("An: ",An)

# NOTE: if op is hermitian, then it's expectation values should be all real

## prepare |0> state ##
state = QuantumCircuit(nqbits)
print("\n STATE \n")
print(state)

## Run estimator ##

estimator = Estimator()
expectation_value = estimator.run(state, op).result().values
##expectation_value = estimator.run(state, op, shots=10000).result().values
print("*********************************************")
print("USING OPERATORS")
print("REAL[ E[<0| U^d An V(k) |0>] ] = ",expectation_value[0].real)
print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",expectation_value[0].imag)
print("*********************************************")

###########################################################################
###########################################################################

## import old code ##

from qva import *

## establish gate and left hand side ##
g = [2,0]
circ_RHS = QuantumCircuit(nqbits+1)
circ_RHS.ch(0,1)
circ_RHS.ch(0,2)


print("*********************************************")
print("USING OUR OLD CODE")
##print("REAL[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Re",parameters,nlayers,circ_RHS))
##print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Im",parameters,nlayers,circ_RHS))

treal = bpsi_hadamard(nqbits,g,"Re",parameters,nlayers,circ_RHS)
timag = bpsi_hadamard(nqbits,g,"Im",parameters,nlayers,circ_RHS)
print("REAL[ E[<0| U^d An V(k) |0>] ] = ",treal)
print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",timag)
print("*********************************************")

uc:       ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     └───┘
U:  Operator([[ 0.5+0.j,  0.5+0.j,  0.5+0.j,  0.5+0.j],
          [ 0.5+0.j, -0.5+0.j,  0.5+0.j, -0.5+0.j],
          [ 0.5+0.j,  0.5+0.j, -0.5+0.j, -0.5+0.j],
          [ 0.5+0.j, -0.5+0.j, -0.5+0.j,  0.5+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
U dagger:  Operator([[ 0.5-0.j,  0.5-0.j,  0.5-0.j,  0.5-0.j],
          [ 0.5-0.j, -0.5-0.j,  0.5-0.j, -0.5-0.j],
          [ 0.5-0.j,  0.5-0.j, -0.5-0.j, -0.5-0.j],
          [ 0.5-0.j, -0.5-0.j, -0.5-0.j,  0.5-0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Is the U-Operator Unitary?  True
Is the U-Operator Hermition?  True
     ┌───────┐ ░    ┌───────┐ ░ 
q_0: ┤ Ry(1) ├─░──■─┤ Ry(3) ├─░─
     ├───────┤ ░  │ ├───────┤ ░ 
q_1: ┤ Ry(2) ├─░──■─┤ Ry(4) ├─░─
     └───────┘ ░    └───────┘ ░ 
     ┌───────┐ ░    ┌───────┐ ░ 
q_0: ┤ Ry(1) ├─░──■─┤ Ry(3) ├─░─
     ├───────┤ ░  │ ├───────┤ ░ 
q_1: ┤ Ry(2) ├─░──■─┤ Ry(4) ├─░─
     └───────┘ ░    └───────┘ ░ 
V:  Operator(

In [5]:



###############################
# <0| V(k)^D An^D Am V(k) |0> #
###############################

nqbits = 2
nlayers = 2

state = QuantumCircuit(nqbits)
parameters = [1, 2, 3, 4]
state = ansatz_RYZ(state, qubits, parameters, nlayers)
print(state)


An = SparsePauliOp.from_list([("XY", 1)])
An = An.adjoint()
An = qi.Operator(An)
print("An: ",An)
Am = SparsePauliOp.from_list([("IY", 1)])
AM = qi.Operator(Am)
print("Am: ", Am)

op = An.compose(Am)
print(op)

estimator = Estimator()
expectation_value = estimator.run(state, op).result().values
print(expectation_value)
##expectation_value = estimator.run(state, op, shots=10000).result().values
print("*********************************************")
print("USING OPERATORS")
print("REAL[ E[<0| V(k)^D Am^D An V(k) |0>] ] = ",expectation_value[0].real)
print("IMAG[ E[<0| V(k)^D Am^D An V(k) |0>] ] = ",expectation_value[0].imag)
print("*********************************************")

## odd code
g = [2,1]
k = [2,0]
print("*********************************************")
print("USING OUR OLD CODE")
##print("REAL[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Re",parameters,nlayers,circ_RHS))
##print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",bpsi_hadamard(nqbits,g,"Im",parameters,nlayers,circ_RHS))

treal = psi_norm_hadamard(nqbits,g,k,"Re",parameters,nlayers)
timag = psi_norm_hadamard(nqbits,g,k,"Im",parameters,nlayers)
print("REAL[ E[<0| U^d An V(k) |0>] ] = ",treal)
print("IMAG[ E[<0| U^d An V(k) |0>] ] = ",timag)
print("*********************************************")

     ┌───────┐ ░    ┌───────┐ ░ 
q_0: ┤ Ry(1) ├─░──■─┤ Ry(3) ├─░─
     ├───────┤ ░  │ ├───────┤ ░ 
q_1: ┤ Ry(2) ├─░──■─┤ Ry(4) ├─░─
     └───────┘ ░    └───────┘ ░ 
An:  Operator([[0.+0.j, 0.+0.j, 0.+0.j, 0.-1.j],
          [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
          [0.+0.j, 0.-1.j, 0.+0.j, 0.+0.j],
          [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
Am:  SparsePauliOp(['IY'],
              coeffs=[1.+0.j])
Operator([[0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
          [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))
[-0.0061912]
*********************************************
USING OPERATORS
REAL[ E[<0| V(k)^D Am^D An V(k) |0>] ] =  -0.006191202889650471
IMAG[ E[<0| V(k)^D Am^D An V(k) |0>] ] =  0.0
*********************************************
*********************************************
USING OUR OLD CODE
       ┌───┐ ░  ░         